## Batch attribute using Signals

This notebook creates a new view using the SDK that will be computed using batch processing using an auto-generated dbt project with the same name.

<div class="alert alert-info">
<b>Please note:</b>
Batch attributes are attributes that are computed in the warehouse. They typically span over 1 or more days which would not be possible to do in stream. 
</div>

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Autogenerated dbt model/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["API_URL"],
    api_key=os.environ["API_KEY"],
    api_key_id=os.environ["API_KEY_ID"],
    org_id=os.environ["ORG_ID"],
)

### Define a new attribute

There are 4 main types of attributes that you may likely want to define: 
1. Actions that happened in the `last_x_number of days`

2. events or properties that happened for the `first time`

3. `last time `

4. `overall aggregation`s. 

We have illustrated each of these 4 types with an example blocks below. 

1. `products_added_to_cart_attribute_last_7_days`: This attribute calculates the number of add to cart ecommerce events in the last 7 days (Please note how the period is noted -> period="P7D").

2. `total_product_price_clv`: This attribute is calculated across the customer lifetime (period left as None)

3. `first_mkt_source`: This attribute takes the first page_view event and reads the mkt_source property for a specific entity (domain_userid)

4. `last_device_class`: This attribute takes the first page_view event and extracts and retrieves the yauaa deviceClass property for a specific entity (domain_userid)

Each block creates a single attribute definition including the logic how it should be calculated (its filters and aggregation).

In [ ]:
from snowplow_signals import (
    Attribute,
    Criteria,
    Criterion,
    Event,
)
from datetime import timedelta

products_added_to_cart_last_7_days = Attribute(
    name="products_added_to_cart_last_7_days",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].name",
    criteria=Criteria(
        all=[
            Criterion(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="=",
                value="add_to_cart",
            ),
        ],
    ),
    period=timedelta(days=7),
)

total_product_price_clv = Attribute(
    name="total_product_price_clv",
    type="float",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="snowplow_ecommerce_action",
            version="1-0-2",
        )
    ],
    aggregation="sum",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1[0].price",
    criteria=Criteria(
        all=[
            Criterion(
                property="unstruct_event_com_snowplowanalytics_snowplow_ecommerce_snowplow_ecommerce_action_1:type",
                operator="=",
                value="add_to_cart"
            )   
        ]
    ),
)

first_mkt_source = Attribute(
    name="first_mkt_source",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="first",
    property="mkt_source",
)

last_device_class = Attribute(
    name="last_device_class",
    type="string",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="last",
    property="contexts_nl_basjes_yauaa_context_1[0]:deviceClass",
)

Now go ahead and rewrite the above example code block to fit your need and add as many attributes as you like that you want to be processed together in the same dbt project and ultimately one attribute table.

<div class="alert alert-info">
<b>Please note:</b>
The name of the attribute will be a column name in the warehouse, be mindful of the warehouse limitations (keep it descriptive but concise). 
</div>

### Wrapping the attribute in a view

All attributes need to be included in views that can be considered as "tables" of attributes which will be processed together in the warehouse using dbt in an incremental fashion to save costs. Views are immutable and versioned. 

Below you can see how you can create a view using the attribute definitions provided earlier.

In [ ]:
from snowplow_signals import View, user_entity

view = View(
    name="batch_ecommerce_attributes",
    version=1,
    entity=user_entity,
    attributes=[
        products_added_to_cart_last_7_days,
        total_product_price_clv,
        first_mkt_source,
        last_device_class
    ],
)

### Testing the view

Execute the view from the atomic events table to verify that it works correctly. To keep the test simple the data will be filtered on the last hour only regardless of the period defined and the results will also be limited per 10 users.

In [ ]:
data = sp_signals.test(
    view,
    app_ids=["website"],
)
data

### Applying the view to Signals

The following block pushes the view definition to the Signals API and makes it available for processing.

In [ ]:
sp_signals.apply([view])

### Generating the dbt project 

Fetch and copy the API URL that you will need to generate the dbt project including the sql models that will incrementally update your attributes in your github repository. 

`filtered_events -> daily_aggregates -> attributes`


In [ ]:
print("URL for the view:", f"{sp_signals.api_client.api_url}/api/v1/registry/views/{view.name}")